# Лабораторная работа

## Студент: Ворошилов Владислав
## Группа: ПИ20-3
## Подгруппа: 2
## Вариант: 3

In [70]:
from sqlalchemy import MetaData, Table, Column, Integer, String, Float, create_engine, select, and_, or_, not_, text, desc
from sqlalchemy.sql import func, distinct, exists
from sqlalchemy import union, union_all


## Создание БД и таблицы

In [127]:
path_to_data = ".."



In [72]:
engine = create_engine(f'sqlite+pysqlite:///{path_to_data}/Data/BikeshareServise.sqlite')
metadata = MetaData()
metadata.reflect(bind=engine)
bikeshare = Table(
    "BikeShare", metadata,
    Column("ride_id", String, primary_key = True), 
    Column("rideable_type", String), 
    Column("started_at", String), 
    Column("ended_at", String), 
    Column("start_station_name", String), 
    Column("start_station_id", Integer), 
    Column("end_station_name", String), 
    Column("end_station_id", Integer),
    Column("start_lat", Float),
    Column("start_lng", Float),
    Column("end_lat", Float),
    Column("end_lng", Float),
    Column("member_casual", String),
    Column("duration", Integer),
    extend_existing=True
)

bikeshare.create(engine, checkfirst=True)


## Вставка данных из файла 

In [78]:
import pandas as pd
path_to_file = path_to_data + "/Data/"
file_name = "BikeShare_05_06.csv"
path = path_to_file + file_name

In [79]:
csv_data = pd.read_csv(path,sep=',')

In [80]:
data_list = csv_data.to_dict('records')

In [81]:
conn = engine.connect()
result = conn.execute(bikeshare.insert(), data_list)

In [77]:
# Удаление всех строк из таблицы
result = conn.execute(bikeshare.delete())

## SQL запросы к БД

### 1. Покажите длительность с накопительным итогом по поездкам, начавшимся в интервале с 9:00 до 12:00 в порядке возрастания времени начала поездки.

In [129]:
subq = select(bikeshare.c.ride_id, bikeshare.c.started_at, (func.strftime("%s", bikeshare.c.ended_at) - func.strftime("%s",bikeshare.c.started_at)).label("duration"))
subq = subq.select_from(bikeshare).subquery()

window_subq = select(func.sum(subq.c.duration).over(order_by=subq.c.started_at).label("sum_duration"), subq.c.started_at, subq.c.ride_id, subq.c.duration)
window_subq = window_subq.select_from(subq).subquery()

stmt = select(window_subq.c.ride_id, window_subq.c.started_at, window_subq.c.duration, window_subq.c.sum_duration)
stmt = stmt.where(func.time(window_subq.c.started_at).between("09:00:00", "12:00:00"))
print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
for row in result:
    print(f"ride: {row.ride_id}, started:{row.started_at}, current_dutation: {row.duration}, sum: {row.sum_duration}")
    

SELECT anon_1.ride_id, anon_1.started_at, anon_1.duration, anon_1.sum_duration 
FROM (SELECT sum(anon_2.duration) OVER (ORDER BY anon_2.started_at) AS sum_duration, anon_2.started_at AS started_at, anon_2.ride_id AS ride_id, anon_2.duration AS duration 
FROM (SELECT "BikeShare".ride_id AS ride_id, "BikeShare".started_at AS started_at, strftime(:strftime_1, "BikeShare".ended_at) - strftime(:strftime_2, "BikeShare".started_at) AS duration 
FROM "BikeShare") AS anon_2) AS anon_1 
WHERE time(anon_1.started_at) BETWEEN :time_1 AND :time_2
82
ride: D93BCF7A196DF00E, started:2022-03-05 09:42:30, current_dutation: 3615, sum: 3615
ride: 490D366D402EE243, started:2022-03-05 09:42:33, current_dutation: 275, sum: 3890
ride: 256BEED882DFFC89, started:2022-03-05 09:45:30, current_dutation: 2804, sum: 6694
ride: 8CFED920BF0EB622, started:2022-03-05 09:45:36, current_dutation: 3425, sum: 10119
ride: 5556CCA0DA83A7B5, started:2022-03-05 09:46:19, current_dutation: 263, sum: 10382
ride: 371E416B7C5664C5

### 2. Покажите среднее время продолжительности поездки по каждой паре станций отправления и назначения.

In [130]:
subq = select(bikeshare.c.ride_id, bikeshare.c.start_station_name, bikeshare.c.end_station_name, (func.strftime("%s", bikeshare.c.ended_at) - func.strftime("%s",bikeshare.c.started_at)).label("duration"))
subq = subq.select_from(bikeshare).subquery()

stmt = select(subq.c.start_station_name.distinct(), subq.c.end_station_name, func.round(func.avg(subq.c.duration).over(partition_by=[subq.c.start_station_name, subq.c.end_station_name]), 2).label("avg_duration"))
stmt = stmt.select_from(subq)
print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))

print(f"started | ended | avg")
for row in result:
    print(f"{row.start_station_name} | {row.end_station_name} | {row.avg_duration}")
    

SELECT DISTINCT anon_1.start_station_name, anon_1.end_station_name, round(avg(anon_1.duration) OVER (PARTITION BY anon_1.start_station_name, anon_1.end_station_name), :round_1) AS avg_duration 
FROM (SELECT "BikeShare".ride_id AS ride_id, "BikeShare".start_station_name AS start_station_name, "BikeShare".end_station_name AS end_station_name, strftime(:strftime_1, "BikeShare".ended_at) - strftime(:strftime_2, "BikeShare".started_at) AS duration 
FROM "BikeShare") AS anon_1
104
started | ended | avg
10th & G St NW | 10th & G St NW | 3230.86
10th St & L'Enfant Plaza SW | Lincoln Memorial | 2560.0
11th & V st NW | 11th & V st NW | 3840.25
14th & Rhode Island Ave NW | 14th & Rhode Island Ave NW | 746.25
15th & M St NW | 15th & M St NW | 4682.5
15th & P St NW | 15th & W St NW | 261.0
15th & W St NW | 15th & P St NW | 329.83
15th & W St NW | 15th & W St NW | 2425.67
15th St & Constitution Ave NW | Henry Bacon Dr & Lincoln Memorial Circle NW | 1106.0
15th St & Pennsylvania Ave NW | 7th & F St N

### 3. Покажите, сколько в процентах занимает время каждой поездки между начальной и конечной станциями от суммарного времени, затраченного на все поездки между станциями данной пары.

In [117]:
subq = select(bikeshare.c.ride_id, bikeshare.c.start_station_name, bikeshare.c.end_station_name, (func.strftime("%s", bikeshare.c.ended_at) - func.strftime("%s",bikeshare.c.started_at)).label("duration"))
subq = subq.select_from(bikeshare).subquery()

window_subq = select(subq.c.ride_id, subq.c.start_station_name, subq.c.end_station_name, subq.c.duration, func.sum(subq.c.duration).over(partition_by=[subq.c.start_station_name, subq.c.end_station_name]).label("sum_duration"))
window_subq = window_subq.select_from(subq).subquery()

stmt = select(window_subq.c.ride_id, window_subq.c.start_station_name, window_subq.c.end_station_name, window_subq.c.duration, window_subq.c.sum_duration, func.round(window_subq.c.duration*100.0 / window_subq.c.sum_duration, 2).label("part_of_the_sum"))
stmt = stmt.select_from(window_subq)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
for row in result:
    print(f"ride_id: {row.ride_id} | started: {row.start_station_name} | ended: {row.end_station_name} | duration: {row.duration} | sum_duration: {row.sum_duration} |% from sum: {row.part_of_the_sum} %")
    

SELECT anon_1.ride_id, anon_1.start_station_name, anon_1.end_station_name, anon_1.duration, anon_1.sum_duration, round((anon_1.duration * :duration_1) / anon_1.sum_duration, :round_1) AS part_of_the_sum 
FROM (SELECT anon_2.ride_id AS ride_id, anon_2.start_station_name AS start_station_name, anon_2.end_station_name AS end_station_name, anon_2.duration AS duration, sum(anon_2.duration) OVER (PARTITION BY anon_2.start_station_name, anon_2.end_station_name) AS sum_duration 
FROM (SELECT "BikeShare".ride_id AS ride_id, "BikeShare".start_station_name AS start_station_name, "BikeShare".end_station_name AS end_station_name, strftime(:strftime_1, "BikeShare".ended_at) - strftime(:strftime_2, "BikeShare".started_at) AS duration 
FROM "BikeShare") AS anon_2) AS anon_1
460
ride_id: AF60DEA8245BEE9F | started: 10th & G St NW | ended: 10th & G St NW | duration: 542 | sum_duration: 22616 |% from sum: 2.4 %
ride_id: F3EC021DB5B6F898 | started: 10th & G St NW | ended: 10th & G St NW | duration: 6190 |

### 4. Покажите 4 самых долгих поездки с каждой станции отправления упорядоченные по станции отправления и по убыванию длительности поездки для данной станции.

In [126]:
subq = select(bikeshare.c.ride_id, bikeshare.c.start_station_name, bikeshare.c.end_station_name, (func.strftime("%s", bikeshare.c.ended_at) - func.strftime("%s",bikeshare.c.started_at)).label("duration"))
subq = subq.select_from(bikeshare).subquery()

window_subq = select(subq.c.ride_id, subq.c.start_station_name, subq.c.end_station_name, subq.c.duration, func.dense_rank().over(partition_by=[subq.c.start_station_name, subq.c.end_station_name], order_by=desc(subq.c.duration)).label("rank"))
window_subq = window_subq.select_from(subq).subquery()

stmt = select(window_subq.c.ride_id, window_subq.c.start_station_name, window_subq.c.end_station_name, window_subq.c.duration, window_subq.c.rank)
stmt = stmt.select_from(window_subq)
stmt = stmt.where(window_subq.c.rank < 5)
stmt = stmt.order_by(window_subq.c.start_station_name, window_subq.c.end_station_name, window_subq.c.rank)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
for row in result:
    print(f"rank: {row.rank}  | ride_id: {row.ride_id} | started: {row.start_station_name} | ended: {row.end_station_name} | duration: {row.duration}")
    

SELECT anon_1.ride_id, anon_1.start_station_name, anon_1.end_station_name, anon_1.duration, anon_1.rank 
FROM (SELECT anon_2.ride_id AS ride_id, anon_2.start_station_name AS start_station_name, anon_2.end_station_name AS end_station_name, anon_2.duration AS duration, dense_rank() OVER (PARTITION BY anon_2.start_station_name, anon_2.end_station_name ORDER BY anon_2.duration DESC) AS rank 
FROM (SELECT "BikeShare".ride_id AS ride_id, "BikeShare".start_station_name AS start_station_name, "BikeShare".end_station_name AS end_station_name, strftime(:strftime_1, "BikeShare".ended_at) - strftime(:strftime_2, "BikeShare".started_at) AS duration 
FROM "BikeShare") AS anon_2) AS anon_1 
WHERE anon_1.rank < :rank_1 ORDER BY anon_1.start_station_name, anon_1.end_station_name, anon_1.rank
386
rank: 1  | ride_id: F3EC021DB5B6F898 | started: 10th & G St NW | ended: 10th & G St NW | duration: 6190
rank: 2  | ride_id: 0A0AC3ABC8A5A077 | started: 10th & G St NW | ended: 10th & G St NW | duration: 6159
ra

### 5. Разбейти поездки на квантили по длительности, упорядочив по возрастанию длительности.

In [125]:
subq = select(bikeshare.c.ride_id, bikeshare.c.start_station_name, bikeshare.c.end_station_name, (func.strftime("%s", bikeshare.c.ended_at) - func.strftime("%s",bikeshare.c.started_at)).label("duration"))
subq = subq.select_from(bikeshare).subquery()

window_subq = select(subq.c.ride_id, subq.c.start_station_name, subq.c.end_station_name, subq.c.duration, func.ntile(4).over(order_by=subq.c.duration).label("quantile"))
window_subq = window_subq.select_from(subq).subquery()

stmt = select(window_subq.c.ride_id, window_subq.c.start_station_name, window_subq.c.end_station_name, window_subq.c.duration, window_subq.c.quantile)
stmt = stmt.select_from(window_subq)
stmt = stmt.order_by(window_subq.c.quantile)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
for row in result:
    print(f"quantile: {row.quantile} | ride_id: {row.ride_id} | started: {row.start_station_name} | ended: {row.end_station_name} | duration: {row.duration} ")
    

SELECT anon_1.ride_id, anon_1.start_station_name, anon_1.end_station_name, anon_1.duration, anon_1.quantile 
FROM (SELECT anon_2.ride_id AS ride_id, anon_2.start_station_name AS start_station_name, anon_2.end_station_name AS end_station_name, anon_2.duration AS duration, ntile(:ntile_1) OVER (ORDER BY anon_2.duration) AS quantile 
FROM (SELECT "BikeShare".ride_id AS ride_id, "BikeShare".start_station_name AS start_station_name, "BikeShare".end_station_name AS end_station_name, strftime(:strftime_1, "BikeShare".ended_at) - strftime(:strftime_2, "BikeShare".started_at) AS duration 
FROM "BikeShare") AS anon_2) AS anon_1 ORDER BY anon_1.quantile
460
quantile: 1 | ride_id: 54C91AB252D51300 | started: 11th & V st NW | ended: 11th & V st NW | duration: 192 
quantile: 1 | ride_id: 0F221F9E96D0E9CD | started: 4th & Florida Ave NE | ended: 1st & M St NE | duration: 195 
quantile: 1 | ride_id: BC6A581D2121B7E2 | started: 15th & W St NW | ended: 15th & P St NW | duration: 210 
quantile: 1 | ride_